In [1]:
import pandas as pd
import os
import re
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
labeled_data_path = os.path.join(DATA_DIR, "Self_train/self_train_step4.csv")
df = pd.read_csv(labeled_data_path)
len(df)

4121

In [4]:
# 모델과 토크나이저 로드
model_name = 'yanolja/EEVE-Korean-Instruct-10.8B-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(DEVICE)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm):

In [5]:
def extract_quoted_string(text):
    pattern = r'\$\$(.*?)\$\$' 
    matches = re.findall(pattern, text)
    if len(matches) == 0 :
        return "제목없음"
    else :
        return matches[0]

In [ ]:
# 노이즈가 있는 뉴스 기사 제목
ko2en = df.copy()
for tidx, kor_title in enumerate(df['text']) :
    prompt = f"""You are an AI translator proficient in both Korean and English. Please translate the following Korean sentence into English.
                Korean sentence: '{kor_title}'
                
                Please follow these guidelines when translating:
                1. Translate the Korean text into English. While doing so, maintain the original meaning of the sentence and consider the context to provide a natural translation.
                2. If there are any awkward parts in the text, please revise them to make them more natural. 
                3. If there are any incomplete or insufficient parts, please fill them in or expand on them as necessary.
                4. Adjust the translated result to fit the format of a news article headline, making it sound natural.

                Translated headline:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_text = generated_text.split("Translated headline:")[-1].split('\n')[0].strip()
    ko2en.loc[tidx, 'text'] = cleaned_text

ko2en.to_csv(f'./data/cleaning_BT/k2e.csv', index=False)

en2ko = ko2en.copy()
for tidx, en_title in enumerate(ko2en['text']) :
    prompt = f"""당신은 한국어와 영어에 능통한 AI 번역가입니다. 다음 영어 문장을 한국어로 번역해주세요.
                영어 문장: '{en_title}'
                
                번역 시 다음 지침을 따라주세요:
                1. 영어 텍스트를 한국어로 번역하세요. 이 과정에서 문장의 원래 의미를 유지하고 맥락을 고려하여 자연스러운 번역을 제공하세요.
                2. 텍스트에 어색한 부분이 있다면 더 자연스럽게 수정해주세요.
                3. 불완전하거나 부족한 부분이 있다면 필요에 따라 채우거나 확장해주세요.
                4. 번역된 결과를 뉴스 기사 헤드라인 형식에 맞게 조정하여 자연스러운 뉴스기사 제목이 되도록 만드세요.

                번역된 헤드라인:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_text = generated_text.split("번역된 헤드라인:")[-1].split('\n')[0].strip()
    en2ko.loc[tidx, 'text'] = cleaned_text

en2ko.to_csv(f'./data/cleaning_BT/e2k.csv', index=False)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임
cleaned :  "Regime divides over time limit for Paris missile event attendance"


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  찰 국 로한 회장이 맡은 주요 프로젝트
cleaned :  "Chul National Laboratory Chairman's Major Projects"


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...
cleaned :  "Agamez scores 33 points as Woori Card thrashes KB Insurance; 3rd place secured"


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합
cleaned :  "President Park, How Surprised Are You…Visits the Pohang Earthquake Site"


KeyboardInterrupt: 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  KISA's Park Min-jeong, Elected as APTLD Director
cleaned :   KISA 박민정, APTLD 회장으로 당선


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  Real Madrid and Atletico Madrid face off in the UEFA Champions League 4th round Madrid Derby
cleaned :   UEFA 챔피언스리그 4차전 마드리드 더비: 레알 마드리드와 아틀레티코 마드리드 대결


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  Frida Kahlo's Philosophy Is Hard to Understand for the U.S. Ambassador to Mexico
cleaned :   프리


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  8 Injured, 1 Seriously Injured in Western US T3 Mainland
cleaned :   서부 미국 T3 본토에서 8명 부상, 1명 중상
origin :  13 Nova Lines 2Mi Jets 10: Final Prize
cleaned :   13개의 노바 라인 2미 제트 10: 최종 상금


: 